In [12]:
import pandas as pd
import re
import copy

In [13]:
df = pd.read_csv("paragraph_per_topic.csv", index_col=0)
df["topic"] = df["topic"].str.replace("\t", "").str.strip()

rename = [
    ("Сервис и коммуникации", "Сервис и коммуникация"),
    ("Маркетинг, реклама, продажи", "Маркетинг, продажи, реклама"),
    ("Качество и безопасность продукта", "Качество и безопасность продукт"),
    ("Профсоюз и коллективный договор", "Профсоюз и Коллективный договор"),
    ("Экология поставщиков", "Экология"),
    ("Работники поставщиков", "Работники"),
]

for old, new in rename:
    df["topic"] = df["topic"].str.replace(old, new)
df = df.dropna()
df.head()

,topic,text,parsed_text
0,Климат,Масса выбросов ПГ сократилась за счет реализац...,масса выброс пг сократиться счёт реализация ме...
1,Климат,В 2022 г. была разработана Климатическая дорож...,в г. разработать климатический дорожный карта ...
2,Климат,В 2022 году Эн+ продолжила работу по борьбе с ...,в год эн+ продолжить работа борьба изменение к...
3,Климат,Хотя периметр потенциального разделения низко ...,хотя периметр потенциальный разделение низко о...
4,Климат,- активы с относительно низким углеродным след...,актив относительно низкий углеродный следом да...


In [14]:
df["topic"].value_counts().head(20)

Вода                           48
Климат                         46
Энергия                        42
Отходы                         37
Этика                          32
Воздух                         30
Безопасность и охрана труда    30
Здоровье и благополучие        29
Оплата труда                   28
Поставщики в целом             28
Благотворительность            27
Корпоративное управление       27
Малый и локальный бизнес       25
Маркетинг, продажи, реклама    23
Биоразнообразие                23
Риски                          20
Коренные народы                20
Обучение и развитие            19
Волонтерство                   18
Отчетность и прозрачность      17
Name: topic, dtype: int64

In [15]:
words = pd.read_csv("../topic_words.csv.zip")
words.head()

,meta,topic,weight,type,word
0,env,Климат,1,0,ch4
1,env,Климат,1,0,co2
2,env,Климат,1,0,ghg protocol
3,env,Климат,1,0,n2o
4,env,Климат,1,0,scope 1


In [18]:
for topic in words["topic"].unique():
    if topic not in df["topic"].unique():
        print(topic)

Здоровье и благополучие сотрудн


In [19]:
for topic in df["topic"].unique():
    if topic not in words["topic"].unique():
        print(topic)

Рекультивация нарушенных земель
Экологичность продукта
Многообразие, равенство и инклюзивность
Доступность
Поставщики в целом
Права человека


In [20]:
re_dict = {}
sheets_dict = {}
base_dict = {}
importance_dict = {}

for idx, row in words.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
    if row["word"].find(",") != -1:
        continue
    sheets_dict[row["topic"]][row["word"]] = row["weight"]  # row["type"]
    base_dict[row["topic"]][row["word"]] = 0
    importance_dict[row["topic"]] = 0

In [21]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile(
        "((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))"
    )

In [22]:
re_advanced_dict = {}

for key, val in sheets_dict.items():
    advanced_words = [word for word, weight in val.items() if weight > 1]
    re_advanced_dict[key] = re.compile(
        "((?<!\w)" + "(?!\w)|(?<!\w)".join(advanced_words) + "(?!\w))"
    )

In [25]:
result = []

for idx, row in df.iterrows():
    text = row["parsed_text"]
    if row["topic"] not in sheets_dict.keys():
        result.append(
            {
                "topic": row["topic"],
                "original_text": row["text"],
                "parsed_text": text,
                "percent_base_words": None,
                "percent_advanced_words": None,
                "base_words": None,
                "advanced_words": None,
            }
        )
        continue

    found_base_words = list(set(re_dict[row["topic"]].findall(text)))
    found_advanced_words = list(set(re_advanced_dict[row["topic"]].findall(text)))
    total_words = len(text.split(" "))
    result.append(
        {
            "topic": row["topic"],
            "original_text": row["text"],
            "parsed_text": text,
            "percent_base_words": len(found_base_words) / total_words * 100,
            "percent_advanced_words": len(found_advanced_words) / total_words * 100,
            "base_words": copy.deepcopy(found_base_words),
            "advanced_words": copy.deepcopy(found_advanced_words),
        }
    )

In [26]:
results = pd.DataFrame(result)
results.to_csv("paragraph_topic_score.csv")
# df.join(results).to_csv("paragraph_topic_score.csv")